# AES Differential Side-channel Analysis

Side-channel is an domain that allows attacks on the implementation of a cryptosystem. We will put the **DPA
attack** into practice to ﬁnd the master key of an **AES with a vulnerable implementation**.

**Aims**
- Find a leak
- Develop & practice the DPA on AES
- Retriev the master key

**Plan**
1. Import packages
2. Loading & plotting data
3. Find a leak with reverse
4. DPA attack with your own selection function
5. Develop your own DPA attack
6. Conclusion

<img src='images/DoIt.png' style='width: 100px'>

# ONE SOLUTION

#### Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
    
plt.rcParams["figure.figsize"] = [20, 5]

---
<img src='images/DoIt.png' style='width: 100px'>

## Import the traces needed for the analysis

#### 🫵 Your turn: Load the trace 'AES_traces_set_1st_round.ets'
from SideSCA-Traces-Public: git clone https://github.com/BeneoLix/SideSCA-Traces-Public.git

In [ ]:
import estraces

In [ ]:
ths = estraces.read_ths_from_ets_file("../SideSCA-Traces-Public/AES_traces_set_1st_round.ets") 

---
<img src='images/DoIt.png' style='width: 100px'>

#### 🫵 Your turn: Print the ths information, the ths len

❓Questions
- What information is stored in *ths*?
- For each data item, what is the data type? What is its size? 

💻[numpy documentation](https://numpy.org/devdocs/user/absolute_beginners.html)

💻[Estrace documentation](https://eshard.gitlab.io/estraces/estraces.html)

> 💡**Tips**: *ths* (TraceHeaderSet) class dedicated to trace and trace set manipulation from ETS file

In [ ]:
#obtain stored informations
print(ths)

In [ ]:
# collect traces, plaintext, key and ciphertext

In [ ]:
# Select array named "plaintext"
plaintext_np_array = ths.plaintext
print(type(plaintext_np_array))
print(plaintext_np_array)
#Indicates the dimensions of an np.array
plaintext_np_array.shape

In [ ]:
# collect traces, plaintext, key and ciphertext
plaintext_np_array

In [ ]:
plaintext_np_array = ths.plaintext
print(type(plaintext_np_array))
print(plaintext_np_array)
plaintext_np_array.shape

In [ ]:
ciphertext_np_array = ths.ciphertext
print(type(ciphertext_np_array))
print(ciphertext_np_array)
ciphertext_np_array.shape

In [ ]:
key_np_array = ths.key
print(type(key_np_array))
print(key_np_array)
key_np_array.shape

In [ ]:
traces_np_array = ths.samples
print(type(traces_np_array))
print(traces_np_array)
traces_np_array.shape

---
<img src='images/DoIt.png' style='width: 100px'>

## Plot and Observe  the traces

#### 🫵 Your turn: Plot the first trace

💻[matplotlib documentation](https://numpy.org/devdocs/user/absolute_beginners.html)

❓Questions
- Do you recognize AES in the trace?


In [ ]:
plt.plot(ths[0].samples.T)
plt.show()

----------
<img src='images/DoIt.png' style='width: 100px'>

## Develop your own DPA attack

### Locating a DPA leak in an AES

This attack technique needs to perform guesses on a part of the key (**divide-and-conquer approach**) and for each guess, knowing the plaintext (resp. the ciphertext) we can compute the intermediate data that is manipulated by the targeted device during the computation. 

Of course only the right guess will lead the values really manipulated by the device.

In that case the DPA attack will succeed and gives the information the guess leading to highest DPA peak matches with the secret key.

<img src="./images/AESscheme.png" width="200px">


#### 🫵 Your turn: Using the SCAred library, perform a DPA reverse and plot the result to locate the vulnerable operation.

In [ ]:
import scared

In [ ]:
def aes_function_firstRde_SBout(plaintext, key):
    return scared.aes.encrypt(plaintext = plaintext, key = key, at_round=1, after_step=scared.aes.base.Steps.SUB_BYTES)

In [ ]:
container = scared.Container(ths)

In [ ]:
Selection_function = scared.reverse_selection_function(aes_function_firstRde_SBout)

In [ ]:
Reverse_SB = scared.DPAReverse(selection_function = Selection_function, model = scared.Monobit(0))
Reverse_SB.run(container)

In [ ]:
Reverse_SB.results.shape

#### 🫵 Your turn:  Plot results

In [ ]:
plt.rcParams['figure.figsize']=(20,3)
plt.plot(Reverse_SB.results.T)
plt.show()

❓ Questions

1. What is a selection function? Which operation is targeted?
1. What is a fault model? Which one is used?
1. Is the targeted operation vulnerable to a DPA attack? Why?

-------------
<img src='images/DoIt.png' style='width: 100px'>

### Perform the Attack on the first round

You've identified the operation that's leaking, let's attack it!

So develop your own DPA selection function 

Knowing the plaintexts (AES inputs) we know the value given as input to the first AES round.

Performing guesses on each key byte, we can guess all the intermediate values of the first round and and use these values with DPA to exploit the related side-channel traces collected and then perform the attack.

It can be the output of the key addition, the output of the SubBytes operations.

---
#### 🫵 Your turn: Code your own selection function: output of SubBytes at first round

In [ ]:
import numpy as _np

SBOX = _np.array([
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16],
    dtype=_np.uint8)

Version given as example

In [ ]:
def my_select_funct_Sbox_out(data, guesses):
    
    
    
    
            
    return ...

--------
#### 🫵 Your turn: Test it

In [ ]:
S_matrix = my_select_funct_Sbox_out(ths.plaintext, range(256))

In [ ]:
print(S_matrix.shape)

<img src='images/DoIt.png' style='width: 100px'>

### Use this selection function with scared as distinguisher for DPA

We will validate our implementation by using our selection function with scared 

---
#### 🫵 Your turn: Define it as the selection function to use in attack object

In [ ]:
@scared.attack_selection_function
def sel(plaintext, guesses):
    return my_select_funct_Sbox_out(plaintext, guesses)

---
<img src='images/DoIt.png' style='width: 100px'>

#### 🫵 Your turn: Set the leakage model, the distinguisher and discrimant

In [ ]:
M = scared.Monobit(0)  #here we select the bit Oth bit the internal values guessed.
# DPA peaks can be positive and negatives, considering the best score requires to take the max value in absolute value.
# The discriminant to use is then: 
d = scared.maxabs

---
#### 🫵 Your turn: set the attack

In [ ]:
# define for scared the traces selected as ths earlier
# playing with the number of traces in the ths we can see how many traces are needed

#container = scared.Container(ths)
container = scared.Container(ths[:1000])

In [ ]:
attack_dpa = scared.DPAAttack(
                selection_function=sel, 
                model=M, 
                discriminant=d)

---
<img src='images/DoIt.png' style='width: 100px'>

#### 🫵 Your turn: Run it

In [ ]:
attack_dpa.run(container)

#### 🫵 Your turn: Compare the found key with the ths key

❓ Questions

1. Are the keys the same?

In [ ]:
found_key = attack_dpa.scores.argmax(0).squeeze()

In [ ]:
vhex = np.vectorize(hex)
print("the key found is:   ", vhex(found_key))
print("the correct key is: ", vhex(ths[0].key))


In [ ]:
np.equal(found_key, ths[0].key)

You can play with the number of traces to use less traces than the full length of ths from now.

We can observe < 1000 traces are enough to recover the key.

---
<img src='images/DoIt.png' style='width: 100px'>

### Print and plot the result

In [ ]:
attack_dpa.results.shape

### Plot the result


In [ ]:
plt.rcParams["figure.figsize"] = [20, 3]
for key_byte_nb in range(16):
    plt.plot(attack_dpa.results.T[:, key_byte_nb, :])
plt.show()

Plot the DPA traces for each key byte, we can also highlight the correct key byte in different color to identify it vs. bad guesses

**We also observe the interesting are in the trace is in the window [1100:1700] and that attacking in this area could be sufficient to recover the key.**

It also corresponds to the SubBytes are in the original collected trace(s) of the AES execution if we look on the power trace(s) collected.

**It might then be relevant to attack in this reduced area of the traces to improve the attack efficiency.**

In [ ]:
plt.rcParams["figure.figsize"] = [16, 2]
for key_byte_nb in range(16):
    for i in range(256):
        plt.plot(attack_dpa.results[i, key_byte_nb, :], color='lightgrey')
    plt.plot(attack_dpa.results[ths[0].key[key_byte_nb], key_byte_nb, :], color='red')
    plt.title('DPA result for byte '+str(key_byte_nb))
    plt.show()

-------------
<img src='images/DoIt.png' style='width: 100px'>

### Develop your own DPA attack
#### Doing your own distinguisher for DPA

__🫵 Your turn:__ Do your own **Difference of Mean** (DoM) on traces for a given guess and selection function